In [1]:
import geopandas as gpd 
import pandas as pd

from shapely import wkt

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
DEFAULT_CRS = "EPSG:4326"

# read building data
O poligono utilizado para realizar download de dados de construção do Google Open Building foi: POLYGON((-46.92166552116743 -23.35298450738664,-46.19244799187055 -23.35298450738664,-46.19244799187055 -23.899033072097712,-46.92166552116743 -23.899033072097712,-46.92166552116743 -23.35298450738664))

In [4]:

# buildings_sp = gpd.read_parquet("spBuilding.geoparquet") # lendo arquivo do OverTure
builds_csv = pd.read_csv("data/open_buildings_v3_polygons_sp.csv")


In [5]:
builds_csv['geometry'] = builds_csv['geometry'].apply(wkt.loads)
buildings_sp = gpd.GeoDataFrame(builds_csv, geometry='geometry', crs=DEFAULT_CRS)


In [10]:
#buildings_sp.rename(columns={'area_in_meters':'area'}, inplace=True)
buildings_sp.head()

,latitude,longitude,area_in_meters,confidence,geometry,full_plus_code
0,-23.536363,-46.198169,339.8561,0.8957,"POLYGON ((-46.19806 -23.53642, -46.19811 -23.5...",588MFR72+FP4G
1,-23.552310,-46.199052,28.7468,0.6921,"POLYGON ((-46.19902 -23.55233, -46.19903 -23.5...",588MCRX2+39GF
2,-23.534467,-46.205638,215.6351,0.9199,"POLYGON ((-46.20555 -23.53452, -46.20557 -23.5...",588MFQ8V+6P98
3,-23.507791,-46.198459,58.1231,0.6945,"POLYGON ((-46.19841 -23.50781, -46.19844 -23.5...",588MFRR2+VJM7
4,-23.558505,-46.208676,122.6622,0.7789,"POLYGON ((-46.20863 -23.55849, -46.20863 -23.5...",588MCQRR+HGWV


# Reading h2 

In [6]:
hex_sp = gpd.read_file("data/shapeFiles/wgs84_hex_grid_sp_v2")

In [7]:
hex_sp.head()

,fid,id_hex,abbrev_mun,name_muni,code_muni,geometry
0,2563.0,89a81009a8bffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.43596 -23.58661, -46.43800 -23.5..."
1,2564.0,89a8108dca7ffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.77886 -23.89706, -46.78089 -23.8..."
2,2565.0,89a81015a8bffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.74000 -23.72290, -46.74204 -23.7..."
3,2566.0,89a810019d7ffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.62778 -23.65457, -46.62982 -23.6..."
4,2567.0,89a8100d9d7ffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.62335 -23.50435, -46.62539 -23.5..."


# Intersect data

In [8]:
if buildings_sp.crs != DEFAULT_CRS:
    buildings_sp = buildings_sp.to_crs(DEFAULT_CRS)
if hex_sp.crs != DEFAULT_CRS:
    hex_sp = hex_sp.to_crs(DEFAULT_CRS)

In [9]:
building_join_h3 =  buildings_sp.overlay(hex_sp[['fid', 'geometry']], how='intersection')


In [10]:

building_join_h3["area"] = (
    building_join_h3
    .geometry
    .to_crs("EPSG:5641")
    .area) # Area may be invalid for a geographic CRS using degrees as units; use GeoSeries.to_crs() to project geometries to a planar CRS before using this function.


In [11]:
building_join_h3.head()

,latitude,longitude,area_in_meters,confidence,full_plus_code,fid,geometry,area
0,-23.581625,-46.636729,131.5081,0.8565,588MC997+88RW,16594.0,"POLYGON ((-46.63673 -23.58170, -46.63680 -23.5...",155.843947
1,-23.663600,-46.654067,84.7656,0.7678,588M88PW+H93X,10987.0,"POLYGON ((-46.65411 -23.66364, -46.65410 -23.6...",100.578102
2,-23.489794,-46.636847,48.2780,0.8282,588MG967+37J9,8654.0,"POLYGON ((-46.63685 -23.48984, -46.63689 -23.4...",57.131702
3,-23.592264,-46.498459,25.1553,0.7473,588MCG52+3JVF,3411.0,"POLYGON ((-46.49850 -23.59227, -46.49849 -23.5...",29.815231
4,-23.491956,-46.658361,51.0183,0.7954,588MG85R+6M6M,5790.0,"POLYGON ((-46.65841 -23.49195, -46.65834 -23.4...",53.346947


In [17]:
building_join_h3_na = building_join_h3.dropna(subset = ['fid'])

In [18]:
area_sum_by_hex = building_join_h3.groupby(['fid']).agg({
    "area": ['sum', 'mean', 'median', 'std', 'var', 'count']
}).reset_index()

In [19]:
area_sum_by_hex.columns = ['_'.join(col).strip() for col in area_sum_by_hex.columns.values if col != 'fid']


In [23]:
area_sum_by_hex.head()

,fid_,area_sum,area_mean,area_median,area_std,area_var,area_count
0,2563.0,13769.277107,52.554493,43.123784,40.794578,1664.197557,262
1,2564.0,1502.033709,150.203371,67.706910,174.333301,30392.099715,10
2,2567.0,61092.031359,168.762518,75.959449,391.982562,153650.328875,362
3,2569.0,103.110937,20.622187,17.462186,10.122136,102.457642,5
4,2570.0,33317.493661,87.447490,71.875638,63.973600,4092.621448,381


In [27]:
hex_area = hex_sp.join(area_sum_by_hex.set_index('fid_'), how='left', rsuffix='sum', on='fid')

In [29]:
hex_area['area_sum'] = hex_area['area_sum'].fillna(0)
hex_area['area_mean'] = hex_area['area_mean'].fillna(0)
hex_area['area_median'] = hex_area['area_median'].fillna(0)
hex_area['area_std'] = hex_area['area_std'].fillna(0)
hex_area['area_var'] = hex_area['area_var'].fillna(0)
hex_area['area_count'] = hex_area['area_count'].fillna(0)




In [30]:
hex_area['area_hex'] =  (
    hex_area
    .geometry
    .to_crs("EPSG:5641")
    .area) # Area may be invalid for a geographic CRS using degrees as units; use GeoSeries.to_crs() to project geometries to a planar CRS before using this function.
hex_area["area_const"] = hex_area['area_sum']/ hex_area['area_hex']

In [31]:
hex_area.head()

,fid,id_hex,abbrev_mun,name_muni,code_muni,geometry,area_sum,area_mean,area_median,area_std,area_var,area_count,area_hex,area_const
0,2563.0,89a81009a8bffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.43596 -23.58661, -46.43800 -23.5...",13769.277107,52.554493,43.123784,40.794578,1664.197557,262.0,126099.311483,0.109194
1,2564.0,89a8108dca7ffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.77886 -23.89706, -46.78089 -23.8...",1502.033709,150.203371,67.706910,174.333301,30392.099715,10.0,125814.918518,0.011938
2,2565.0,89a81015a8bffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.74000 -23.72290, -46.74204 -23.7...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,125897.010258,0.000000
3,2566.0,89a810019d7ffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.62778 -23.65457, -46.62982 -23.6...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,125979.202351,0.000000
4,2567.0,89a8100d9d7ffff,spo,Sao Paulo,3550308.0,"POLYGON ((-46.62335 -23.50435, -46.62539 -23.5...",61092.031359,168.762518,75.959449,391.982562,153650.328875,362.0,126033.393325,0.484729


In [25]:
hex_area.isna().sum()

fid            0
id_hex         0
abbrev_mun     0
name_muni      0
code_muni      0
geometry       0
area_sum       0
area_mean      0
area_median    0
area_std       0
area_var       0
area_count     0
area_hex       0
area_const     0
dtype: int64

Final transformations

In [32]:
hex_area["fid"] = hex_area["fid"].astype(int)

In [27]:
hex_area.to_file("data/WGS84_hex_construcao_open_buildings_2023.gpkg", driver="GPKG")